In [4]:
from gurobipy import *
import gurobipy as gp

# type declaration
from typing import Dict, List


import pandas as pd

# %load_ext nb_black
# %load_ext lab_black

In [6]:
combinations_oil_refineries_distributioncenters: Dict[tuple, int] = {
    ("T", "N", "P"): 11 + 11,
    ("T", "C", "P"): 7 + 7,
    ("T", "S", "P"): 2 + 5,
    ("T", "N", "A"): 11 + 7,
    ("T", "C", "A"): 7 + 4,
    ("T", "S", "A"): 2 + 3,
    ("CA", "N", "P"): 7 + 11,
    ("CA", "C", "P"): 4 + 7,
    ("CA", "S", "P"): 8 + 5,
    ("CA", "N", "A"): 7 + 7,
    ("CA", "C", "A"): 4 + 4,
    ("CA", "S", "A"): 8 + 3,
}


# quantity constraint
combinations_oil_refineries_distributioncenters_list = list(
    combinations_oil_refineries_distributioncenters.keys()
)


oil_fields: List[str] = ["T", "CA"]
refineries: List[str] = ["NO", "C", "S"]
distribution_centers: List[str] = ["P", "A"]


# oil_field_limits
oil_field_limits: Dict[str, int] = {"CA": 50000, "T": 10000}


# demand limits
demand_limits: Dict[str, int] = {"P": 20000, "A": 25000}


# set the decision function
model = gp.Model()
model.ModelSense = gp.GRB.MINIMIZE

# set the objective function
obj = {}
for combi in combinations_oil_refineries_distributioncenters_list:
    obj[combi] = model.addVar(
        lb=0,
        obj=combinations_oil_refineries_distributioncenters[combi],
        name="{}".format(str(combi)).replace(" ", ""),
    )

# CONSTRAINT FOR SUPPLY: essenetially: for every combintion with T at beginning; this is <= 10000; and <= for CA
for oil_field in oil_field_limits.keys():
    lhs = 0
    for objective_r in obj.keys():
        if objective_r[0] == oil_field:
            lhs += obj[objective_r]
    model.addConstr(
        lhs <= oil_field_limits[oil_field], name="oil_limit_{}".format(oil_field)
    )

# Constraint for demand
for distro in demand_limits.keys():
    lhs = 0
    for objective_r in obj.keys():
        if objective_r[-1] == distro:
            lhs += obj[objective_r]
    model.addConstr(lhs == demand_limits[distro], name="demand_{}".format(distro))

model.optimize()
if not model.status == gp.GRB.OPTIMAL:
    print("something went wrong")
print("optimal value", model.objval)
model.printAttr("X")

Set parameter Username
Academic license - for non-commercial use only - expires 2023-11-10
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (linux64)

CPU model: 11th Gen Intel(R) Core(TM) i7-1165G7 @ 2.80GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 4 rows, 12 columns and 24 nonzeros
Model fingerprint: 0x74384aec
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e+00, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+04, 5e+04]
Presolve removed 4 rows and 12 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.8000000e+05   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Optimal objective  3.800000000e+05
optimal value 380000.0

    Variable            X 
-------------------------
('T','S','P')        10000 
(

In [7]:
print("Sensitivity Analysis:")
model.printAttr(["X", "Obj", "SAObjLow", "SAObjUP"])
model.printAttr(["RC", "LB", "SALBLow", "SALBUp", "UB", "SAUBLow", "SAUBUp"])
model.printAttr(["Sense", "Slack", "Pi", "RHS", "SARHSLow", "SARHSUp"])

Sensitivity Analysis:

    Variable            X          Obj     SAObjLow      SAObjUP 
----------------------------------------------------------------
('T','N','P')            0           22            7          inf 
('T','C','P')            0           14            7          inf 
('T','S','P')        10000            7         -inf            8 
('T','N','A')            0           18            4          inf 
('T','C','A')            0           11            4          inf 
('T','S','A')            0            5            4          inf 
('CA','N','P')            0           18           11          inf 
('CA','C','P')        10000           11           10           13 
('CA','S','P')            0           13           11          inf 
('CA','N','A')            0           14            8          inf 
('CA','C','A')        25000            8         -inf            9 
('CA','S','A')            0           11            8          inf 

    Variable           RC          

## Q 3

In [8]:
from gurobipy import *
import gurobipy as gp

# type declaration
from typing import Dict, List


import pandas as pd

# %load_ext nb_black
%load_ext lab_black

ModuleNotFoundError: No module named 'lab_black'

In [9]:
stores = ["A", "B"]

demand = {
    ("A", 0): 45,
    ("A", 1): 20,
    ("A", 2): 20,
    ("A", 3): 25,
    ("A", 4): 15,
    ("A", 5): 28,
    ("A", 6): 15,
    ("B", 0): 8,
    ("B", 1): 12,
    ("B", 2): 23,
    ("B", 3): 30,
    ("B", 4): 12,
    ("B", 5): 10,
    ("B", 6): 33,
}

days = 7

model = gp.Model()
model.ModelSense = gp.GRB.MINIMIZE

# purchasing day one
Purc_A = model.addVars(days, name="Purc_A", obj=200)
Purc_B = model.addVars(days, name="Purc_B", obj=200)

# "C allocation (5)" from A to B and B to A
cab = model.addVars(days, name="Cab", obj=5)
cba = model.addVars(days, name="Cba", obj=5)

# "E allocation (20)" from A to B and B to A
eab = model.addVars(days, name="Eab", obj=20)
eba = model.addVars(days, name="Eba", obj=20)

# set the function
model.setObjective(
    gp.quicksum(
        ((Purc_A[t] + Purc_B[t]) * 200 + (eab[t] + eba[t]) * 20 + (cab[t] + cba[t]) * 5)
        for t in range(days)
    )
)

Inventory_A = model.addVars(days, name="Inventory_A")
Inventory_B = model.addVars(days, name="Inventory_B")

## inventory management A
# day 1 purchase only; index 0
Inventory_A[0] = Purc_A[0]
# day 2 no purcahse but E allocation: inventory day 2 = Inventory day 1 (t - 1 = 0) + transfers from b at
# day t - 1 - minus all transfers from a to b and from a to b in the 5 € concerns
Inventory_A[1] = Inventory_A[0] + eba[0] - eab[0] - cab[0]
# day 3 and beyond; is essentially the same as above but now you can also receive
for t in range(2, days):
    Inventory_A[t] = (
        Inventory_A[t - 1] + eba[t - 1] + cba[t - 2] - eab[t - 1] - cab[t - 1]
    )

## the same for B just the other way around!
Inventory_B[0] = Purc_B[0]
Inventory_B[1] = Inventory_B[0] + eab[0] - eba[0] - cba[0]
for t in range(2, days):
    Inventory_B[t] = (
        Inventory_B[t - 1] + eab[t - 1] + cab[t - 2] - eba[t - 1] - cba[t - 1]
    )


# enforce that inventory is at least as big as demand on date t for both inventory a and b
for t in range(days):
    model.addConstr(Inventory_A[t] >= demand[("A", t)], name="Inventory_A")

for t in range(days):
    model.addConstr(Inventory_B[t] >= demand[("B", t)], name="Inventory_B")


# only day one can be used for purchasesing
for t in range(1, days):
    lhs = Purc_A[t] + Purc_B[t]
    model.addConstr(lhs == 0)

# only the last day does not work in this case as there cant be two day transfers (the C transfers)
model.addConstr((cab[days - 1] + cba[days - 1]) == 0)


model.optimize()

print("Total Cost:", model.OBJVAL)
model.printAttr(["x"])

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (linux64)

CPU model: 11th Gen Intel(R) Core(TM) i7-1165G7 @ 2.80GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 21 rows, 56 columns and 184 nonzeros
Model fingerprint: 0x16a8f63d
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e+00, 2e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [8e+00, 4e+01]
Presolve removed 9 rows and 32 columns
Presolve time: 0.00s
Presolved: 12 rows, 24 columns, 166 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.0680000e+04   1.250000e+01   0.000000e+00      0s
       6    1.1265000e+04   0.000000e+00   0.000000e+00      0s

Solved in 6 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.126500000e+04
Total Cost: 11265.0

    Variable            x 
-------------------------
   Purc_A[0]           45 
   Purc_B[0]           

In [10]:
print("Sensitivity Analysis:")
model.printAttr(["X", "Obj", "SAObjLow", "SAObjUP"])
model.printAttr(["RC", "LB", "SALBLow", "SALBUp", "UB", "SAUBLow", "SAUBUp"])
model.printAttr(["Sense", "Slack", "Pi", "RHS", "SARHSLow", "SARHSUp"])

Sensitivity Analysis:

    Variable            X          Obj     SAObjLow      SAObjUP 
----------------------------------------------------------------
   Purc_A[0]           45          200          180          inf 
   Purc_A[1]            0          200            0          inf 
   Purc_A[2]            0          200            0          inf 
   Purc_A[3]            0          200            0          inf 
   Purc_A[4]            0          200            0          inf 
   Purc_A[5]            0          200            0          inf 
   Purc_A[6]            0          200            0          inf 
   Purc_B[0]           10          200           45          220 
   Purc_B[1]            0          200            0          inf 
   Purc_B[2]            0          200            0          inf 
   Purc_B[3]            0          200            0          inf 
   Purc_B[4]            0          200            0          inf 
   Purc_B[5]            0          200            0   

In [11]:
model.write("problem.lp")
with open("problem.lp") as f:
    print(f.read())

\ LP format - for model browsing. Use MPS format to capture full model detail.
Minimize
  200 Purc_A[0] + 200 Purc_A[1] + 200 Purc_A[2] + 200 Purc_A[3]
   + 200 Purc_A[4] + 200 Purc_A[5] + 200 Purc_A[6] + 200 Purc_B[0]
   + 200 Purc_B[1] + 200 Purc_B[2] + 200 Purc_B[3] + 200 Purc_B[4]
   + 200 Purc_B[5] + 200 Purc_B[6] + 5 Cab[0] + 5 Cab[1] + 5 Cab[2]
   + 5 Cab[3] + 5 Cab[4] + 5 Cab[5] + 5 Cab[6] + 5 Cba[0] + 5 Cba[1]
   + 5 Cba[2] + 5 Cba[3] + 5 Cba[4] + 5 Cba[5] + 5 Cba[6] + 20 Eab[0]
   + 20 Eab[1] + 20 Eab[2] + 20 Eab[3] + 20 Eab[4] + 20 Eab[5] + 20 Eab[6]
   + 20 Eba[0] + 20 Eba[1] + 20 Eba[2] + 20 Eba[3] + 20 Eba[4] + 20 Eba[5]
   + 20 Eba[6] + 0 Inventory_A[0] + 0 Inventory_A[1] + 0 Inventory_A[2]
   + 0 Inventory_A[3] + 0 Inventory_A[4] + 0 Inventory_A[5]
   + 0 Inventory_A[6] + 0 Inventory_B[0] + 0 Inventory_B[1]
   + 0 Inventory_B[2] + 0 Inventory_B[3] + 0 Inventory_B[4]
   + 0 Inventory_B[5] + 0 Inventory_B[6]
Subject To
 Inventory_A: Purc_A[0] >= 45
 Inventory_A: Purc_A[0]

### 3c

In [14]:
df = pd.read_csv(
    "/home/angelo/Downloads/sails.csv"
)

In [15]:
dict_A = {}
dict_B = {}
for index, row in df.iterrows():
    dict_A[("A", row["day"] - 1)] = row["demandA"]

for index, row in df.iterrows():
    dict_B[("B", row["day"] - 1)] = row["demandB"]

dictionary_ab = dict_A | dict_B

In [16]:
stores = ["A", "B"]
demand = dictionary_ab
days = 365

In [25]:
model = gp.Model()
model.ModelSense = gp.GRB.MINIMIZE

# purchasing day one
Purc_A = model.addVars(days, name="Purc_A", obj=200)
Purc_B = model.addVars(days, name="Purc_B", obj=200)

# "C allocation (5)" from A to B and B to A
cab = model.addVars(days, name="Cab", obj=5)
cba = model.addVars(days, name="Cba", obj=5)

# "E allocation (20)" from A to B and B to A
eab = model.addVars(days, name="Eab", obj=20)
eba = model.addVars(days, name="Eba", obj=20)

# set the function
model.setObjective(
    gp.quicksum(
        ((Purc_A[t] + Purc_B[t]) * 200 + (eab[t] + eba[t]) * 20 + (cab[t] + cba[t]) * 5)
        for t in range(days)
    )
)


Inventory_A = model.addVars(days, name="Inventory_A")
Inventory_B = model.addVars(days, name="Inventory_B")

## inventory management A
# day 1 purchase only; index 0
Inventory_A[0] = Purc_A[0]
# day 2 no purcahse but E allocation: inventory day 2 = Inventory day 1 (t - 1 = 0) + transfers from b at day t - 1 - minus all transfers from a to b and from a to b in the 5 € concerns
Inventory_A[1] = Inventory_A[0] + eba[0] - eab[0] - cab[0]
# day 3 and beyond; is essentially the same as above but now you can also receive
for t in range(2, days):
    Inventory_A[t] = (
        Inventory_A[t - 1] + eba[t - 1] + cba[t - 2] - eab[t - 1] - cab[t - 1]
    )

## the same for B just the other way around!
Inventory_B[0] = Purc_B[0]
Inventory_B[1] = Inventory_B[0] + eab[0] - eba[0] - cba[0]
for t in range(2, days):
    Inventory_B[t] = (
        Inventory_B[t - 1] + eab[t - 1] + cab[t - 2] - eba[t - 1] - cba[t - 1]
    )


# enforce that inventory is at least as big as demand on date t for both inventory a and b
for t in range(days):
    model.addConstr(Inventory_A[t] >= demand[("A", t)], name="Inventory_A")

for t in range(days):
    model.addConstr(Inventory_B[t] >= demand[("B", t)], name="Inventory_B")

# only day one can be used for purchasesing
for t in range(1, days):
    lhs = Purc_A[t] + Purc_B[t]
    model.addConstr(lhs == 0)

# only the last day does not work in this case as there cant be two day transfers (the C transfers)
model.addConstr((cab[days - 1] + cba[days - 1]) == 0)


model.optimize()

if not model.status == gp.GRB.OPTIMAL:
    print("something went wrong")
print("optimal value", model.objval)
model.printAttr("X")

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (linux64)

CPU model: 11th Gen Intel(R) Core(TM) i7-1165G7 @ 2.80GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1095 rows, 2920 columns and 532172 nonzeros
Model fingerprint: 0xfd7ccca5
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e+00, 2e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 7e+01]
Presolve removed 367 rows and 1464 columns
Presolve time: 0.13s
Presolved: 728 rows, 1456 columns, 531438 nonzeros

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 2.646e+05
 Factor NZ  : 2.654e+05 (roughly 3 MB of memory)
 Factor Ops : 1.289e+08 (less than 1 second per iteration)
 Threads    : 3

Barrier performed 0 iterations in 0.24 seconds (0.13 work units)
Barrier solve interrupted - model solved by another algori

In [18]:
dict_3c = {}
dict_3c[("A", 0)] = Inventory_A[0].X
for day in range(1, 365):
    if day == 1:
        dict_3c[("A", day)] = (
            dict_3c[("A", day - 1)] + eba[day - 1].X - eab[day - 1].X - cab[day - 1].X
        )
    else:
        dict_3c[("A", day)] = (
            dict_3c[("A", day - 1)]
            + eba[day - 1].X
            - eab[day - 1].X
            + cba[day - 2].X
            - cab[day - 1].X
        )

In [19]:
dict_3cb = {}
dict_3cb[("B", 0)] = Inventory_B[0].X
for day in range(1, 365):
    if day == 1:
        dict_3cb[("B", day)] = (
            dict_3cb[("B", day - 1)] + eab[day - 1].X - eba[day - 1].X - cba[day - 1].X
        )
    else:
        dict_3cb[("B", day)] = (
            dict_3cb[("B", day - 1)]
            + eab[day - 1].X
            - eba[day - 1].X
            + cab[day - 2].X
            - cba[day - 1].X
        )

In [20]:
# how many sails did you transport in total using each type of transportation
# service?
# eab
dict_eab = {}
for day in range(365):
    dict_eab[day] = eab[day].X

# eba
dict_eba = {}
for day in range(365):
    dict_eba[day] = eba[day].X

# cab
dict_cab = {}
for day in range(365):
    dict_cab[day] = cab[day].X
# cba
dict_cba = {}
for day in range(365):
    dict_cba[day] = cba[day].X

In [21]:
sum(list(dict_eab.values()))

10.0

In [22]:
sum(list(dict_eba.values()))

9.0

In [23]:
sum(list(dict_cab.values()))

104.0

In [24]:
sum(list(dict_cba.values()))

110.0

In [ ]:
print("has")

In [28]:
print("Sensitivity Analysis:")
model.printAttr(["X", "Obj", "SAObjLow", "SAObjUP"])
model.printAttr(["RC", "LB", "SALBLow", "SALBUp", "UB", "SAUBLow", "SAUBUp"])
model.printAttr(["Sense", "Slack", "Pi", "RHS", "SARHSLow", "SARHSUp"])

Sensitivity Analysis:

    Variable            X          Obj     SAObjLow      SAObjUP 
----------------------------------------------------------------
   Purc_A[0]           58          200          195          205 
   Purc_A[1]            0          200            0          inf 
   Purc_A[2]            0          200            0          inf 
   Purc_A[3]            0          200            0          inf 
   Purc_A[4]            0          200            0          inf 
   Purc_A[5]            0          200            0          inf 
   Purc_A[6]            0          200            0          inf 
   Purc_A[7]            0          200            0          inf 
   Purc_A[8]            0          200            0          inf 
   Purc_A[9]            0          200            0          inf 
  Purc_A[10]            0          200            0          inf 
  Purc_A[11]            0          200            0          inf 
  Purc_A[12]            0          200            0   